<a href="https://colab.research.google.com/github/steiningerjakob/Project-YODA/blob/master/DC_GAN_jakob.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from tensorflow import keras
import os

In [ ]:
# mount google drive

In [58]:
INPUT_DIR = '/content/drive/MyDrive/images_for_gan/'
OUTPUT_DIR = '/content/drive/MyDrive/simple_gan/run_4_batch-size_16_100k-epochs'
N_CHANNELS = 3
LATENT_DIM = 100
IMAGE_HEIGHT = 128
IMAGE_WIDTH = 128
BATCH_SIZE = 16
N_EPOCHS = 100_000

In [59]:
def load_lego_data():
    data_generator = keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=keras.applications.xception.preprocess_input,
        data_format='channels_last',
        rotation_range=0,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=False,
        vertical_flip=False)

    flow_from_directory_params = {'target_size': (IMAGE_HEIGHT, IMAGE_WIDTH),
                                'color_mode': 'grayscale' if N_CHANNELS == 1 else 'rgb',
                                'class_mode': None,
                                'batch_size': BATCH_SIZE}

    image_generator = data_generator.flow_from_directory(
        directory=os.path.join(INPUT_DIR, '.'),
        **flow_from_directory_params)
    
    return image_generator

In [60]:
def get_image_batch(image_generator):
    img_batch = image_generator.next()

    if len(img_batch) != BATCH_SIZE:
        img_batch = image_generator.next()

    assert img_batch.shape == (BATCH_SIZE, IMAGE_HEIGHT,
     IMAGE_WIDTH, N_CHANNELS), img_batch.shape
    return img_batch

In [61]:
def make_generator():
    generator = keras.models.Sequential()
    # Foundation for 4x4 image
    generator.add(keras.layers.Dense(256*4*4, input_dim=LATENT_DIM))
    generator.add(keras.layers.LeakyReLU(alpha=0.2))
    generator.add(keras.layers.Reshape((4, 4, 256)))
    # Upsample to 8x8
    generator.add(keras.layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', use_bias=False))
    generator.add(keras.layers.LeakyReLU(alpha=0.2))
    # Upsample to 16x16
    generator.add(keras.layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', use_bias=False))
    generator.add(keras.layers.LeakyReLU(alpha=0.2))
    # Upsample to 32x32
    generator.add(keras.layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', use_bias=False))
    generator.add(keras.layers.LeakyReLU(alpha=0.2))
    # Upsample to 64x64
    generator.add(keras.layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', use_bias=False))
    generator.add(keras.layers.LeakyReLU(alpha=0.2))
    # Upsample to 128x128
    generator.add(keras.layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', use_bias=False))
    generator.add(keras.layers.LeakyReLU(alpha=0.2))
    # Output 128x128
    generator.add(keras.layers.Conv2D(N_CHANNELS, (3,3), activation='tanh', padding='same', use_bias=False))
    return generator

In [62]:
def make_discriminator():
    discriminator = keras.models.Sequential()
    # Normal
    discriminator.add(keras.layers.Conv2D(64, (3,3), padding='same', input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, N_CHANNELS)))
    discriminator.add(keras.layers.LeakyReLU(alpha=0.2))
    # Downsample to 64x64
    discriminator.add(keras.layers.Conv2D(64, (3,3), strides=(2,2), padding='same'))
    discriminator.add(keras.layers.LeakyReLU(alpha=0.2))
    # Downsample to 32x32
    discriminator.add(keras.layers.Conv2D(64, (3,3), strides=(2,2), padding='same'))
    discriminator.add(keras.layers.LeakyReLU(alpha=0.2))
    # Downsample to 16x16
    discriminator.add(keras.layers.Conv2D(64, (3,3), strides=(2,2), padding='same'))
    discriminator.add(keras.layers.LeakyReLU(alpha=0.2))
    # Downsample to 8x8
    discriminator.add(keras.layers.Conv2D(64, (3,3), strides=(2,2), padding='same'))
    discriminator.add(keras.layers.LeakyReLU(alpha=0.2))
    # Classifier
    discriminator.add(keras.layers.Dropout(0.3))
    discriminator.add(keras.layers.Flatten())
    discriminator.add(keras.layers.Dense(1, activation='sigmoid'))
    opt = keras.optimizers.RMSprop(lr=0.0001, clipvalue=1.0, decay=1e-8)
    discriminator.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return discriminator

In [63]:
def train_gan(gan, generator, discriminator, image_generator):
    # Run training loop for n_epochs
    for epoch in range(N_EPOCHS):
        real_images = get_image_batch(image_generator)
        real_labels = np.ones((BATCH_SIZE, 1))
        d_loss1, _ = discriminator.train_on_batch(real_images, real_labels)
        
        random_latent_vectors = np.random.normal(size=(BATCH_SIZE, LATENT_DIM))
        fake_data = generator.predict(random_latent_vectors)
        fake_labels = np.zeros((BATCH_SIZE, 1))
        d_loss2, _ = discriminator.train_on_batch(fake_data, fake_labels)
        
        random_latent_vectors = np.random.normal(size=(BATCH_SIZE, LATENT_DIM))
        misleading_labels = np.ones((BATCH_SIZE, 1))
        g_loss = gan.train_on_batch(random_latent_vectors, misleading_labels)
        
        print('Epoch %d, d1=%.3f, d2=%.3f g=%.3f' % (epoch+1, d_loss1, d_loss2, g_loss))
        
        # Save GAN weights, a fake and a real image every 50 epochs
        if (epoch) % 100 == 0:
            gan.save_weights(os.path.join(OUTPUT_DIR, 'gan.h5'))
            print(f'Saving batch of generated images at adversarial step: {epoch}.')
            random_latent_vectors = np.random.normal(size=(BATCH_SIZE, LATENT_DIM))
            fake_images = generator.predict(random_latent_vectors)
            real_images = get_image_batch(image_generator)
            img = keras.preprocessing.image.array_to_img(fake_images[0])
            img.save(os.path.join(OUTPUT_DIR, 'generated_lego_epoch_' + str(epoch) + '.png'))
            img = keras.preprocessing.image.array_to_img(real_images[0])
            img.save(os.path.join(OUTPUT_DIR, 'real_lego_epoch_' + str(epoch) + '.png'))

In [64]:
def main():
    # Make the GAN
    generator = make_generator()
    discriminator = make_discriminator()
    discriminator.trainable = False
    gan = keras.models.Sequential([generator, discriminator])

    # Compile the GAN
    opt = keras.optimizers.RMSprop(lr=0.0008, clipvalue=1.0, decay=1e-8)
    gan.compile(optimizer=opt, loss='binary_crossentropy')

    # Initialize the data generator
    image_generator = load_lego_data()

    # Train the GAN
    train_gan(gan, generator, discriminator, image_generator)

    # return the GAN
    return train_gan

In [ ]:
trained_gan = main()

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Found 481 images belonging to 37 classes.
Epoch 1, d1=0.706, d2=0.695 g=0.693
Saving batch of generated images at adversarial step: 0.
Epoch 2, d1=0.660, d2=0.696 g=0.692
Epoch 3, d1=0.628, d2=0.762 g=0.654
Epoch 4, d1=0.604, d2=0.995 g=0.642
Epoch 5, d1=0.609, d2=0.823 g=0.633
Epoch 6, d1=0.590, d2=0.874 g=0.620
Epoch 7, d1=0.562, d2=0.967 g=0.564
Epoch 8, d1=0.596, d2=1.002 g=0.530
Epoch 9, d1=0.583, d2=1.063 g=0.484
Epoch 10, d1=0.617, d2=1.072 g=0.475
Epoch 11, d1=0.589, d2=1.080 g=0.464
Epoch 12, d1=0.556, d2=1.122 g=0.445
Epoch 13, d1=0.568, d2=1.117 g=0.459
Epoch 14, d1=0.556, d2=1.148 g=0.450
Epoch 15, d1=0.568, d2=1.146 g=0.430
Epoch 16, d1=0.565, d2=1.177 g=0.415
Epoch 17, d1=0.548, d2=1.186 g=0.429
Epoch 18, d1=0.529, d2=1.191 g=0.416
Epoch 19, d1=0.562, d2=1.095 g=0.467
Epoch 20, d1=0.544, d2=1.025 g=0.470
Epoch 21, d1=0.526, d2=1.073 g=0.471
Epoch 22, d1=0.537, d2=1.133 g=0.428
Epoch 23, d1=0.558, d2=1.158 g=0.422
Epoch 24, d1=0.555, d2=1.130 g=0.424
Epoch 25, d1=0.534, d2